# Human Activity Recognition Using WiFi Signals

## Overview
Human Activity Recognition (HAR) using WiFi signals leverages the unique properties of wireless channel variations to detect different activities.

## Data Format
- **WiFi signal data** is similar to image data in structure, represented in the shape `(channels, height, width)`, but with a different interpretation:
  - `channels` → **channel**
  - `height` → **Time Steps**
  - `width` → **Antenna Pairs (transmitter-receiver combinations)**
- **Labels** represent a predefined set of classes, as is typical in classification tasks.

# Reading Data

In [25]:
!pip install -U -q gdown

import gdown

link = "https://drive.google.com/file/d/17Vfiu90uYeeRqmW-QbhocgBt69mrrScA/view?usp=sharing"
file_id = link.split('/d/')[1].split('/')[0]

# Construct the direct download URL
download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
output_file = "WiFiSensingDataset.pt.zip"
gdown.download(download_url, output_file, quiet=False)

FileURLRetrievalError: Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=17Vfiu90uYeeRqmW-QbhocgBt69mrrScA

but Gdown can't. Please check connections and permissions.

In [ ]:
!unzip WiFiSensingDataset.pt.zip

Archive:  WiFiSensingDataset.pt.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of WiFiSensingDataset.pt.zip or
        WiFiSensingDataset.pt.zip.zip, and cannot find WiFiSensingDataset.pt.zip.ZIP, period.


In [38]:
# Note: We're using torch modules (datasets, dataloaders) to download dataset and easily make batches.
# The NN will be made in numpy and every step will be implemented ourselves

# %%
# Import necessary libraries.
import numpy as np

# Import MNIST dataset and DataLoader from Torchvision. This allows us to easily load and work with the MNIST dataset.
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

# Import the 'to_tensor' function for converting PIL images to tensors. This is necessary because PyTorch works with tensors.
from torchvision.transforms.functional import to_tensor

# Import Matplotlib's pyplot for visualizations. We'll use this to display images and plots.
import matplotlib.pyplot as plt

In [26]:
import torch
import kagglehub

path = kagglehub.dataset_download("jawadkc66/q2-kaust-2025")

print("Path to dataset files:", path)

data = torch.load('/root/.cache/kagglehub/datasets/jawadkc66/q2-kaust-2025/versions/1/WiFiSensingDataset.pt')

100%|██████████| 204M/204M [00:05<00:00, 37.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jawadkc66/q2-kaust-2025/versions/1


<ipython-input-26-7c3c69014005>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('/root/.cache/kagglehub/datasets/jawadkc66/q2-kaust-2025/versions/1/WiFiSe

In [37]:
import torch  # Import the PyTorch library

import torch.nn as nn  # Import the neural network module from PyTorch

from torchvision.datasets import MNIST  # Import MNIST dataset from Torchvision
from torch.utils.data import DataLoader  # Import DataLoader for batching
from torch.optim import Adam  # Import the Adam optimizer

from torchvision.transforms.functional import to_tensor  # Import to_tensor for image conversion

import matplotlib.pyplot as plt  # Import Matplotlib for visualization
data
import pandas as pd
data


{'X_test': tensor([[[[0.4063, 0.1338, 0.4588,  ..., 0.6775, 0.7083, 0.6615],
           [0.3715, 0.2196, 0.4792,  ..., 0.6935, 0.6967, 0.6641],
           [0.3742, 0.2473, 0.4946,  ..., 0.7042, 0.7097, 0.6790],
           ...,
           [0.5245, 0.4429, 0.3430,  ..., 0.7161, 0.7224, 0.6850],
           [0.4956, 0.4228, 0.3719,  ..., 0.7076, 0.6951, 0.6784],
           [0.4861, 0.3396, 0.3864,  ..., 0.7166, 0.7392, 0.6788]]],
 
 
         [[[0.5485, 0.5999, 0.6332,  ..., 0.7803, 0.7650, 0.7257],
           [0.5426, 0.5947, 0.6169,  ..., 0.7856, 0.7601, 0.7224],
           [0.5658, 0.6009, 0.6354,  ..., 0.7856, 0.7708, 0.7141],
           ...,
           [0.5661, 0.6048, 0.6422,  ..., 0.7963, 0.7836, 0.7276],
           [0.5395, 0.6129, 0.6306,  ..., 0.7933, 0.7771, 0.7327],
           [0.5523, 0.5882, 0.6299,  ..., 0.7900, 0.7739, 0.7293]]],
 
 
         [[[0.3518, 0.4836, 0.4677,  ..., 0.8025, 0.7917, 0.7716],
           [0.4130, 0.4547, 0.4947,  ..., 0.8142, 0.7906, 0.7653],
        

# Task 1: Analyze the Dataset ( Stored in `data`)

1. **Determine the number of unique labels** in the dataset.  

2. **Determine the shape of the input data** (number of samples and features).  

3. **Find the maximum value** in the dataset.  

4. **Find the minimum value** in the dataset.  

In [34]:
len(set(data))  #number of unique labels
data

4

# Task 2: Build and Evaluate a Neural Network

1. **Design a Neural Network (Maximum 5 Layers)**  
   Build a compact neural network with no more than 5 layers. Clearly specify the type of each layer (e.g., Dense, Conv2D) and any activation functions used.

2. **Evaluate Your Model**  
   Train your network on the provided dataset and report the evaluation metrics (e.g., accuracy, loss). Discuss the performance of your model and any challenges faced during training.


In [39]:
ni=5
nh=10
no=10
w1=np.random.randn(nh,ni)
b1=np.zeros((nh,1))
w2=np.random.randn(no,nh)
b2=np.zeros((no,1))
w3=np.random.randn((no,nh))
b3=np.zeros((no,1))
w4=np.random.randn((no,nh))
b4=np.zeros((no,1))
w5=np.random.randn((no,nh))
b5=np.zeros((no,1))
num_epochs = 12              # Number of training epochs
lr = 1e-2                    # Learning rate for gradient descent
train_epoch_losses = []      # List to store training loss for each epoch

total_train_imgs = len(train_loader.dataset)

In [ ]:
epoch_loss = 0  # Initialize total loss for the epoch
correct = 0    # Initialize count of correctly classified images

for X, labels in train_loader:  # Iterate over batches of data

    # Preprocess the data
    X = X.numpy().reshape(-1, 28*28).T  # Convert images to numpy arrays and reshape (transpose)
    labels = labels.numpy()           # Convert labels to numpy arrays

    # One-hot encode the labels
    y = np.zeros((no, X.shape[1]))       # Create a matrix of zeros with shape (no, batch_size)
    y[labels, np.arange(y.shape[1])] = 1  # Set the corresponding element to 1 for each label

    # Forward pass
    Z1 = W1 @ X + b1   # Calculate weighted sum and add bias for the first layer
    A1 = sigmoid(Z1)   # Apply sigmoid activation to get the output of the first layer
    Z2 = W2 @ A1 + b2   # Calculate weighted sum and add bias for the second layer
    yhat = sigmoid(Z2)  # Apply sigmoid activation to get the predicted probabilities

    # Calculate loss
    loss = cross_entropy_loss(yhat, y)  # Calculate the cross-entropy loss
    epoch_loss += loss * len(labels)   # Accumulate the loss for the batch

    # Calculate accuracy
    pred = np.argmax(yhat, axis=0)    # Get the predicted class labels
    correct += np.sum(labels == pred)  # Count the number of correct predictions

# Calculate average epoch loss
epoch_loss /= total_train_imgs
train_epoch_losses.append(epoch_loss)  # Store the epoch loss

# Print the loss and accuracy for the epoch
print(f'loss = {epoch_loss}. {correct}/{total_train_imgs} correctly labelled.')

In [ ]:
for i in range(num_epochs):  # Loop over the specified number of epochs

    epoch_loss = 0  # Initialize total loss for the epoch
    correct = 0    # Initialize count of correctly classified images

    for X, labels in train_loader:  # Iterate over batches of data

        # Preprocess the data (same as before)
        X = X.numpy().reshape(-1, 28*28).T
        labels = labels.numpy()

        y = np.zeros((no, X.shape[1]))
        y[labels, np.arange(y.shape[1])] = 1

        # Forward pass (same as before)
        Z1 = W1 @ X + b1
        A1 = sigmoid(Z1)

        Z2 = W2 @ A1 + b2
        yhat = sigmoid(Z2)

        # --------------------
        # Backward pass
        # --------------------

        # Calculate gradients
        dZ2 = yhat - y                                  # Gradient of loss w.r.t Z2
        dW2 = dZ2 @ A1.T                                # Gradient of loss w.r.t W2
        db2 = np.sum(dZ2, axis=1, keepdims=True)         # Gradient of loss w.r.t b2

        dZ1 = W2.T @ dZ2 * A1 * (1 - A1)                # Gradient of loss w.r.t Z1
        dW1 = dZ1 @ X.T                                # Gradient of loss w.r.t W1
        db1 = np.sum(dZ1, axis=1, keepdims=True)         # Gradient of loss w.r.t b1

        # --------------------
        # Optimization (update weights and biases)
        # --------------------
        W2 -= lr * dW2
        b2 -= lr * db2
        W1 -= lr * dW1
        b1 -= lr * db1

        # Calculate classification error (same as before)
        pred = np.argmax(yhat, axis=0)
        correct += np.sum(labels == pred)

        loss = cross_entropy_loss(yhat, y)
        epoch_loss += loss * len(labels)

    # Calculate average epoch loss and print results (same as before)
    epoch_loss /= total_train_imgs
    train_epoch_losses.append(epoch_loss)

    print(f'Epoch {i}, loss = {epoch_loss}. {correct}/{total_train_imgs} correctly labelled.')

Good luck in the exam x)

Prepared by: Ahmed Y. Radwan
